In [19]:
import requests
from bs4 import BeautifulSoup
from functools import reduce
url='https://books.toscrape.com/'
response=requests.get(url)

In [20]:
soup=BeautifulSoup(response.text,'html.parser')

In [21]:
lst = (soup.find('ul', attrs = {'class':''}).find_all('li'))

In [22]:
links = []
for n in range(len(lst)):
    link = soup.find('ul', attrs = {'class':''}).find_all('li')[n].find('a').get('href')
    domain = url
    full_link = domain+link
    links.append(full_link)
links

['https://books.toscrape.com/catalogue/category/books/travel_2/index.html',
 'https://books.toscrape.com/catalogue/category/books/mystery_3/index.html',
 'https://books.toscrape.com/catalogue/category/books/historical-fiction_4/index.html',
 'https://books.toscrape.com/catalogue/category/books/sequential-art_5/index.html',
 'https://books.toscrape.com/catalogue/category/books/classics_6/index.html',
 'https://books.toscrape.com/catalogue/category/books/philosophy_7/index.html',
 'https://books.toscrape.com/catalogue/category/books/romance_8/index.html',
 'https://books.toscrape.com/catalogue/category/books/womens-fiction_9/index.html',
 'https://books.toscrape.com/catalogue/category/books/fiction_10/index.html',
 'https://books.toscrape.com/catalogue/category/books/childrens_11/index.html',
 'https://books.toscrape.com/catalogue/category/books/religion_12/index.html',
 'https://books.toscrape.com/catalogue/category/books/nonfiction_13/index.html',
 'https://books.toscrape.com/catalogue

In [24]:
import csv
headings = ['Book_name','Price','Rate','Category']
with open('books_list.csv','w',encoding='utf-8',newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headings)
    writer.writeheader()
rate_dict = {'One':1, 'Two' : 2 , 'Three' : 3 , 'Four' : 4 , 'Five' : 5}
for url in links:
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    for book in soup.find_all('article',attrs={'class':'product_pod'}):
        pricex=float(book.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
        titlex=book.find('h3').find('a').get('title').strip()
        ratex=rate_dict[book.find('p').get('class')[1]]
        categoryx=soup.find('div',attrs={'class':'page-header action'}).find('h1').get_text()
        with open('books_list.csv', 'a', encoding='utf-8', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=headings)
            writer.writerow({'Book_name':titlex,'Price':pricex,'Rate':ratex,'Category':categoryx})
    while soup.find('li',attrs={'class':'next'}):
        new_page_url=reduce(lambda a,b:a+'/'+b,url.split('/')[:-1])+'/'+soup.find('li',attrs={'class':'next'}).find('a').get('href')
        soup=BeautifulSoup(requests.get(new_page_url).text,'html.parser')
        for book in soup.find_all('article',attrs={'class':'product_pod'}):
            pricex=float(book.find('div',attrs={'class':'product_price'}).find('p').get_text()[2:])
            titlex=book.find('h3').find('a').get('title').strip()
            ratex=rate_dict[book.find('p').get('class')[1]]
            categoryx=soup.find('div',attrs={'class':'page-header action'}).find('h1').get_text()
            with open('books_list.csv', 'a', encoding='utf-8', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=headings)
                writer.writerow({'Book_name':titlex,'Price':pricex,'Rate':ratex,'Category':categoryx})

In [25]:
import pandas as pd
df=pd.read_csv('books_list.csv')
df

,Book_name,Price,Rate,Category
0,It's Only the Himalayas,45.17,2,Travel
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,4,Travel
2,See America: A Celebration of Our National Par...,48.87,3,Travel
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,2,Travel
4,Under the Tuscan Sun,37.33,3,Travel
...,...,...,...,...
995,Why the Right Went Wrong: Conservatism--From G...,52.65,4,Politics
996,Equal Is Unfair: America's Misguided Fight Aga...,56.86,1,Politics
997,Amid the Chaos,36.58,1,Cultural
998,Dark Notes,19.19,5,Erotica


In [26]:
df.shape

(1000, 4)